# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mount isa,-20.7333,139.5000,15.87,72,75,0.00,AU,1689863957
1,1,portland,45.5234,-122.6762,18.36,81,20,1.54,US,1689863361
2,2,whitehorse,60.7161,-135.0538,12.12,68,75,1.03,CA,1689863957
3,3,bilibino,68.0546,166.4372,12.53,80,98,0.78,RU,1689863957
4,4,longyearbyen,78.2186,15.6401,12.91,50,20,6.69,SJ,1689863958


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", frame_width=700, frame_height=500,
                                     size = "Humidity",
                                     color = "City")


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_df = city_data_df.loc[(city_data_df["Max Temp"] < 30) &
                                (city_data_df["Max Temp"] > 20)&
                                (city_data_df["Wind Speed"] < 5)]

# Drop any rows with null values
clean_ideal_temp_df = ideal_temp_df.dropna()

# Display sample data
clean_ideal_temp_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,13,georgetown,5.4112,100.3354,28.97,87,20,1.54,MY,1689863845
16,16,corner brook,48.9667,-57.9484,24.09,87,100,1.14,CA,1689863956
19,19,puerto ayora,-0.7393,-90.3518,25.98,96,81,4.02,EC,1689863960
22,22,bethel,41.3712,-73.4140,27.55,67,40,1.54,US,1689863939
23,23,ribeira grande,38.5167,-28.7000,22.22,94,40,2.24,PT,1689863960


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_ideal_temp_df[["City", "Country", "Lat","Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
13,georgetown,MY,5.4112,100.3354,87,
16,corner brook,CA,48.9667,-57.9484,87,
19,puerto ayora,EC,-0.7393,-90.3518,96,
22,bethel,US,41.3712,-73.4140,67,
23,ribeira grande,PT,38.5167,-28.7000,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
limit = 10
categories = "accommodation.hotel"
params = { "limit": limit,
    "categories": categories,  
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

Starting hotel search
georgetown - nearest hotel: Page 63 hostel
corner brook - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
bethel - nearest hotel: Hampton Inn Danbury
ribeira grande - nearest hotel: Hospedaria JSF
vorgashor - nearest hotel: София
ishikawa - nearest hotel: サンセットヒル
saint-paul - nearest hotel: Hôtel d'application
lwowek slaski - nearest hotel: Piast
hawaiian paradise park - nearest hotel: No hotel found
colonia - nearest hotel: Wasserturm Hotel Cologne
tura - nearest hotel: No hotel found
saipan - nearest hotel: Chalan Kanoa Beach Hotel
pangody - nearest hotel: Газпром энерго
camarinas - nearest hotel: O Parranda
salekhard - nearest hotel: No hotel found
hasaki - nearest hotel: 大新旅館
salym - nearest hotel: Каравани
sofo-birnin-gwari - nearest hotel: No hotel found
suzhou - nearest hotel: Overhere Theme Hotel
duekoue - nearest hotel: Hotel Taouli 2
jamestown - nearest hotel: DoubleTree Jamestown
naze - nearest hotel: ビジネ

In [7]:
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
13,georgetown,MY,5.4112,100.3354,87,Page 63 hostel
16,corner brook,CA,48.9667,-57.9484,87,No hotel found
19,puerto ayora,EC,-0.7393,-90.3518,96,Hostal La Mirada De Solitario George
22,bethel,US,41.3712,-73.4140,67,Hampton Inn Danbury
23,ribeira grande,PT,38.5167,-28.7000,94,Hospedaria JSF


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points("Lng", "Lat", geo=True, tiles="OSM", frame_width=700, frame_height=500,
                                     color = "City",
                                   hover_cols = ["Country", "Humidity", "Hotel Name"])


# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Humidity,Hotel Name)